In [72]:
from langgraph.graph import StateGraph,START,END
from pydantic import BaseModel,Field
from dotenv import load_dotenv
from typing import TypedDict
import operator
from langchain_google_genai import ChatGoogleGenerativeAI
import httpx

In [73]:
import os
load_dotenv()
api_key=os.getenv('GEMINI_API_KEY')

In [74]:
model=ChatGoogleGenerativeAI(
    api_key=api_key,
    model='gemini-2.5-flash'
)

In [75]:
class GitHubRepoState(TypedDict):
    username: str
    repos: list[dict]
    languages: list[str]


In [76]:
class GithubClient:
    def __init__(self,token):
        self.token = token
        self.base_url="https://api.github.com/"

    def _headers(self):
        return{
            "Authorization": f"Bearer {self.token}",
            "Accept": "application/vnd.github.v3+json",
        }
    
    def get_proj_repos(self,username):
        response=httpx.get(f"{self.base_url}users/{username}/repos", headers=self._headers())
        response.raise_for_status()
        return response.json()
    
    def get_proj_languages(self,owner,repo):
        response=httpx.get(f"{self.base_url}repos/{owner}/{repo}/languages", headers=self._headers())
        response.raise_for_status()
        return response.json()


In [ ]:
def get_github_repos_details(state: GitHubRepoState):
    client=GithubClient(token=os.getenv('GITHUB_TOKEN'))
    repos=client.get_proj_repos(state["username"])
    languages=set()
    required_repo_data=[]
    for repo in repos:
        readme=httpx.get(f'https://raw.githubusercontent.com/{state["username"]}/{repo["name"]}/main/README.md')
        required_data={
            "repository_name": repo["name"],
            "repository_url":repo['html_url'],
            "repository_language_url":repo['languages_url'],
            "readme": readme
        }
        languages_data=httpx.get(required_data['repository_language_url'], headers=client._headers())
        languages_data.raise_for_status()
        languages.update(languages_data.json().keys())
        required_repo_data.append(required_data)

    return {"repos": required_repo_data,"languages":list(languages)}

In [78]:
graph=StateGraph(GitHubRepoState,
                 merge={
                     "repos": operator.add,
                     "languages": operator.add
                 })
graph.add_node('get_github_repos_details',get_github_repos_details)
graph.add_edge(START,'get_github_repos_details')
graph.add_edge('get_github_repos_details',END)
workflow=graph.compile()


In [79]:
initial_state={"username":"Ashish0243"}
final_state=workflow.invoke(initial_state)
print(final_state)

{'username': 'Ashish0243', 'repos': [{'repository_name': '1strep', 'repository_url': 'https://github.com/Ashish0243/1strep', 'repository_language_url': 'https://api.github.com/repos/Ashish0243/1strep/languages', 'readme': <Response [200 OK]>}, {'repository_name': 'Backend', 'repository_url': 'https://github.com/Ashish0243/Backend', 'repository_language_url': 'https://api.github.com/repos/Ashish0243/Backend/languages', 'readme': <Response [200 OK]>}, {'repository_name': 'Blog', 'repository_url': 'https://github.com/Ashish0243/Blog', 'repository_language_url': 'https://api.github.com/repos/Ashish0243/Blog/languages', 'readme': <Response [404 Not Found]>}, {'repository_name': 'Book-Store-Project', 'repository_url': 'https://github.com/Ashish0243/Book-Store-Project', 'repository_language_url': 'https://api.github.com/repos/Ashish0243/Book-Store-Project/languages', 'readme': <Response [200 OK]>}, {'repository_name': 'care', 'repository_url': 'https://github.com/Ashish0243/care', 'repository